# **1. Extract information about Justin Trudeau for sysdate-1**

In [1]:
import pandas as pd
import requests
import datetime
import urllib.parse
from datetime import datetime
import pyodbc

# **5. Create a log file to check if any issue happened during the process**

In [9]:
import logging

logging.basicConfig(filename=r"C:\Users\Lion\Desktop\scraper.log",level=logging.DEBUG, format = '%(asctime)s - %(message)s',datefmt='%d-%b-%y %H:%M:%S')

In [3]:
logging.info("Extracting information about Justin Trudeau for sysdate-1")
# define parameters for our request to guardian api for sysdate-1
parameters={
    'from-date' : '2020-01-01',
    'to-date' : str(datetime.today())[:10],
    'q' : 'Justin%20AND%20Trudeau%20OR%20JustinTrudeau',
    'format' : 'json',
    'page' : 1,
    'page-size' : 10,
    'api-key' : "Your api key"
}

main_url='https://content.guardianapis.com/search?'

In [4]:
# checking request status and extract data
def extractData(params, main_urladdress):
    url=main_urladdress+ urllib.parse.urlencode(params).replace('%2520', '%20')
    data= requests.get(url).json()
    if data['response']['status']=='ok':
        return data

In [5]:
fullData = extractData(parameters,main_url)

numberofResults= fullData['response']['total']
numberofPages= fullData['response']['pages']
print('Number of results: {}'.format(numberofResults))
print('Number of pages: {}'.format(numberofPages))

Number of results: 892
Number of pages: 90


In [6]:
def appendfullData(params, numPages, fullData):
    for currentpage in range(2, numPages+1):
        parameters['page']= currentpage
        currentData = extractData(parameters,main_url)
        if currentData['response']['results'] is not None:
            fullData['response']['results'].extend(currentData['response']['results'])
        else:
            continue

In [7]:
# extract result of our responses and add them to a list to make a dataframe later
appendfullData(parameters, numberofPages, fullData)

In [8]:
# just made a dataframe out of created list
df = pd.DataFrame(fullData['response']['results'])
logging.info("make a dataframe out of the json information we have exctracted")

In [9]:
df = df.drop(["sectionId",'apiUrl','isHosted','pillarId','pillarName'],axis=1)

In [10]:
df['webPublicationDate'] = pd.to_datetime(df['webPublicationDate'])

In [11]:
df.rename({"webPublicationDate":"Date"},axis=1,inplace=True)

In [12]:
df

,id,type,sectionName,Date,webTitle,webUrl
0,world/2023/feb/08/canadas-justin-trudeau-greet...,article,World news,2023-02-08 01:13:46+00:00,Canada’s Justin Trudeau greets political oppon...,https://www.theguardian.com/world/2023/feb/08/...
1,world/2023/mar/07/canada-pm-justin-trudeau-spe...,article,World news,2023-03-07 00:18:42+00:00,Justin Trudeau to appoint special rapporteur t...,https://www.theguardian.com/world/2023/mar/07/...
2,world/2022/dec/08/sovereignty-act-passed-alber...,article,World news,2022-12-08 16:53:29+00:00,Alberta ‘sovereignty act’ sets province on col...,https://www.theguardian.com/world/2022/dec/08/...
3,world/2023/mar/24/trudeau-biden-asylum-border-...,article,World news,2023-03-24 21:41:14+00:00,Trudeau to announce US-Canada asylum deal afte...,https://www.theguardian.com/world/2023/mar/24/...
4,tv-and-radio/2022/nov/02/drugs-impotence-and-p...,article,Television & radio,2022-11-02 17:57:23+00:00,"Drugs, impotence and punching Justin Trudeau: ...",https://www.theguardian.com/tv-and-radio/2022/...
...,...,...,...,...,...,...
887,world/live/2020/may/26/coronavirus-live-news-c...,liveblog,World news,2020-05-27 00:03:15+00:00,Putin says Russia past peak of outbreak despit...,https://www.theguardian.com/world/live/2020/ma...
888,world/live/2020/apr/10/coronavirus-live-news-g...,liveblog,World news,2020-04-11 01:00:23+00:00,Confirmed worldwide Covid-19 death toll passes...,https://www.theguardian.com/world/live/2020/ap...
889,world/live/2020/apr/14/coronavirus-live-news-c...,liveblog,World news,2020-04-14 23:46:53+00:00,Italy sees lowest increase in Covid-19 infecti...,https://www.theguardian.com/world/live/2020/ap...
890,world/live/2020/may/12/coronavirus-live-news-d...,liveblog,World news,2020-05-12 23:33:59+00:00,"Worldwide Covid-19 deaths pass 290,000 – as it...",https://www.theguardian.com/world/live/2020/ma...


# **2. Save the result as a file in the local folder**

In [13]:
# save the dataframe to a local folder as a csv file
df.to_csv(r"C:\Users\Lion\Desktop\Test.csv",sep=",")
logging.info("save dataframe as a csv file in local folder to put it on the database")

# **3.Read the file and import it into the database using “Bulk Insert”**

*we can import the csv file in database by our hand and then truncate the table to have just columns and then do the bulk inserting by below code*

In [15]:
# Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=OMID\SQLEXPRESS;'
                      'Database=AdventureWorks2012;'
                      'Trusted_Connection=yes;')

# Create a cursor object
cursor = conn.cursor()


# Execute the BULK INSERT command
# bulk_insert_query = "BULK INSERT [AdventureWorks2012].[dbo].[irancell] FROM 'C:\\Users\\Lion\\Desktop\\Test.csv' WITH (FORMAT = 'CSV', FIRSTROW = 2)"
# cursor.execute(bulk_insert_query)
try:
    create_table_query = "CREATE TABLE irancell(column1 int, id ntext,type ntext, sectionName ntext, Date datetime, webTitle ntext, webUrl ntext )"
    truncate_query = "Truncate Table irancell"
    bulk_insert_query = "BULK INSERT [AdventureWorks2012].[dbo].[irancell] FROM 'C:\\Users\\Lion\\Desktop\\Test.csv' WITH (FORMAT = 'CSV', FIRSTROW = 2)"
    cursor.execute(create_table_query)
    cursor.execute(truncate_query)
    cursor.execute(bulk_insert_query)
# if the try raise an error the except would handle it, cause of error is the table have been created before so we just do the bulk insert
except:
    bulk_insert_query = "BULK INSERT [AdventureWorks2012].[dbo].[irancell] FROM 'C:\\Users\\Lion\\Desktop\\Test.csv' WITH (FORMAT = 'CSV', FIRSTROW = 2)"
    truncate_query = "Truncate Table irancell"
    cursor.execute(truncate_query)
    cursor.execute(bulk_insert_query)

# Commit the transaction
conn.commit()

# Close the cursor and connection objects
cursor.close()
conn.close()
logging.info("WE HAVE DONE IT SUCESSFULLY")

![](https://github.com/showmidelo/Irancell_Task/blob/main/ms%20sql%20server%20test%20for%20bulk%20insert.jpg?raw=true)

# **4. Do require a cleansing process on the data**

*check different section names*

In [ ]:
df.groupby(df['sectionName']).size()

sectionName
Environment            200
Television & radio     200
World news            1600
dtype: int64

In [ ]:
df.groupby(df['Date']).size()

webPublicationDate
2022-09-08 19:48:01+00:00    200
2022-11-02 17:57:23+00:00    200
2022-12-08 12:15:00+00:00    200
2022-12-08 16:53:29+00:00    200
2023-02-08 01:13:46+00:00    200
2023-02-12 07:28:59+00:00    200
2023-03-07 00:18:42+00:00    200
2023-03-09 16:16:12+00:00    200
2023-03-24 21:41:14+00:00    200
2023-04-28 18:45:21+00:00    200
dtype: int64

# **How to schedule python script to run every day at an exact time**

1. In the Windows Task Scheduler, create a new task.

2 . In the "General" tab, enter the name of the task and select how often you want it to run.

3. In the "Triggers" tab, select "New."

4. In the "Begin the task" drop-down, select "On a schedule."

5. In the "Settings" section, select "Daily" and enter the time you want the task to run.

6. In the "Actions" tab, select "New."

7. In the "Action" drop-down, select "Start a program."

8. In the "Program/script" field, enter the full path to the Python interpreter, such as "C:\Python27\python.exe."

9. In the "Add arguments (optional)" field, enter the full path to the "task.py" file.

10. Click "OK" to save the task

# __*first step*__

![](https://github.com/showmidelo/Irancell_Task/blob/main/schedule%20pictures/1.jpg?raw=true)

# __*Second step*__

![](https://github.com/showmidelo/Irancell_Task/blob/main/schedule%20pictures/2.jpg?raw=true)

# __*Third step*__

![](https://github.com/showmidelo/Irancell_Task/blob/main/schedule%20pictures/3.jpg?raw=true)

# __*Fourth step*__

![](https://github.com/showmidelo/Irancell_Task/blob/main/schedule%20pictures/4.jpg?raw=true)